In [ ]:
import numpy as np

In [ ]:
iso = [84.72, 86.72, 88.83, 86.28, 84.28]
aniso = [87.46, 80.67, 76, 77.56, 72.91]

In [ ]:
np.mean(iso), np.std(iso)

In [ ]:
np.mean(aniso), np.std(aniso)

In [ ]:
import pykeops
import torch
import math
from pykeops.torch import LazyTensor

device = 'cpu' 

# rounds to the nearest integer (0 decimal)
x = torch.FloatTensor(1000, 1).uniform_(-10, 10)
y = x.data.clone()
x = x.to(device)
y = y.to(device)
x.requires_grad = True
y.requires_grad = True

x_i = LazyTensor(x[:, None])
s1 = x_i.round(0).sum(0)
s2 = torch.sum(torch.round(y))
print("s1 - s2", torch.abs(s1 - s2).item())
assert torch.abs(s1 - s2) < 1e-3, torch.abs(s1 - s2)

s1.backward()
s2.backward()

print("grad_s1 - grad_s2", torch.max(torch.abs(x.grad - y.grad)).item())
assert torch.max(torch.abs(x.grad - y.grad)) < 1e-3

# rounds to 3 decimal places
x = torch.FloatTensor(1000, 1).uniform_(-1, 1)
y = x.data.clone()
x = x.to(device)
y = y.to(device)
x.requires_grad = True
y.requires_grad = True

x_i = LazyTensor(x[:, None])
s1 = x_i.round(3).sum(0)
s2 = torch.sum(torch.round(y * 1e3)*1e-3)
print("s1 - s2", torch.abs(s1 - s2).item())
assert torch.abs(s1 - s2) < 1e-3, torch.abs(s1 - s2)

s1.backward()
s2.backward()

print("grad_s1 - grad_s2", torch.max(torch.abs(x.grad - y.grad)).item())
assert torch.max(torch.abs(x.grad - y.grad)) < 1e-3

In [ ]:
x = torch.linspace(-1, 1, 100)
plt.scatter(x, torch.acos(x), s=5)

In [ ]:
x = torch.linspace(0, 3.14159, 100)
import matplotlib.pyplot as plt

plt.scatter(x, 1 - torch.heaviside(x - 3.14159, torch.ones(1)), s=5)

In [ ]:
l = torch.nn.Identity()
x = torch.rand(16, 3, 100)
torch.allclose(x, l(x))

In [ ]:
def sub_divide(vertices, faces, level):
    vertices /= torch.norm(vertices, dim=1, keepdim=True)
    for _ in range(level):
        sub_faces = torch.stack((faces[:, [0, 1, 2]], faces[:, [2, 0, 1]], faces[:, [1, 2, 0]]), dim=1)
        vertices_1 = vertices[sub_faces[:, :, [0, 1]]].mean(dim=2)
        vertices_1 /= torch.norm(vertices_1, dim=2, keepdim=True)
        vertices_2 = vertices[sub_faces[:, :, [1, 2]]].mean(dim=2)
        vertices_2 /= torch.norm(vertices_2, dim=2, keepdim=True)
        
        N = vertices.shape[0]
        NV1 = vertices_1.shape[0]*vertices_1.shape[1]
        NV2 = vertices_2.shape[0]*vertices_1.shape[1]
        
        sub_faces[:,:,0] = torch.arange(N, N+NV1).reshape(-1, 3)
        sub_faces[:,:,2] = torch.arange(N+NV1, N+NV1+NV2).reshape(-1, 3)
        
        faces = torch.cat((sub_faces, torch.arange(N, N+NV1).reshape(-1,1, 3)), dim=1).reshape(-1, 3)
        vertices = torch.cat((vertices, vertices_1.reshape(-1, 3), vertices_2.reshape(-1, 3)), dim=0)
        
    return vertices.unique(dim=0)

In [ ]:
def xyz2betagamma(x, y, z):
    """
    Returns new tensors corresponding to angle representation from the cartesian representation.

    Args:
        x (FloatTensor): input tensor, i.e. x positions.
        y (FloatTensor): input tensor, i.e. y positions.
        z (FloatTensor): input tensor, i.e. z positions.

    Returns:
        (FloatTensor): output tensor, i.e. alpha rotation about x axis.
        (FloatTensor): output tensor, i.e. beta rotation about y axis.
        (FloatTensor): output tensor, i.e. gamma rotation about z axis.
    """

    beta = torch.stack(
        (
            torch.atan2(-z, -torch.sqrt(x.pow(2) + y.pow(2))),
            torch.atan2(-z, torch.sqrt(x.pow(2) + y.pow(2))),
        ),
        dim=-1,
    )

    gamma = torch.stack((torch.atan2(-y, -x), torch.atan2(y, x)), dim=-1)

    mask = (beta >= -math.pi) & (beta < math.pi) & (gamma >= -math.pi/2) & (gamma < math.pi/2)
    
    return beta[mask], gamma[mask]


In [ ]:
tetrahedron_vertices = torch.tensor([[1, 0, -1/math.sqrt(2)],[-1, 0, -1/math.sqrt(2)], 
                                     [0, 1, 1/math.sqrt(2)],[0, -1, 1/math.sqrt(2)]])
tetrahedron_faces = torch.tensor([[0, 1, 2], [0, 2, 3], [0, 1, 3], [1, 2, 3]])

In [ ]:
octahedron_vertices = torch.tensor([[1., 0., 0.], [0., 1., 0.], [-1., 0., 0.], 
                                    [0., -1., 0.], [0., 0., 1.], [0., 0., -1.]])
octahedron_faces = torch.tensor([[0, 1, 5], [1, 5, 2], [2, 5, 3], [0, 5, 3], 
                                 [0, 3, 4], [0, 1, 4], [1, 2, 4], [2, 3, 4]])

In [ ]:
phi = (1 + math.sqrt(5))/2

icosahedron_vertices = torch.tensor([[phi, 1., 0.], [-phi, 1., 0.], [phi, -1., 0.], [-phi, -1., 0.],
                                     [1., 0., phi], [1., 0., -phi], [-1., 0., phi], [-1., 0., -phi],
                                     [0., phi, 1.], [0., -phi, 1.], [0., phi, -1.], [0., -phi, -1.]])
icosahedron_faces = torch.tensor([[0, 2, 4], [0, 2, 5], [0, 4, 8], [0, 8, 10], [0, 5, 10],
                                  [1, 3, 6], [1, 3, 7], [1, 6, 8], [1, 7, 10], [1, 8, 10],
                                  [2, 4, 9], [2, 5, 11], [2, 9, 11], [3, 7, 11], [3, 9, 11],
                                  [3, 6, 9], [4, 6, 9], [4, 6, 8], [5, 7, 10], [5, 7, 11]])

In [ ]:
X = sub_divide(icosahedron_vertices, icosahedron_faces, 5)
X.shape

In [ ]:
beta, gamma = xyz2betagamma(X[:,0],X[:,1],X[:,2])
beta.shape

In [ ]:
fig = plt.figure(figsize=(8.0, 8.0))

ax = fig.add_subplot(
    111,
    projection="3d",
    xlim=(-1., 1.),
    ylim=(-1., 1.),
    zlim=(-1., 1.)
)

im = ax.scatter(
    X[:, 0],
    X[:, 1],
    X[:, 2]
)

In [ ]:
N = X.shape[0]
dist = (X.unsqueeze(0) - X.unsqueeze(1)).pow(2).sum(dim=2) + 100 * torch.eye(N, N).bool()
min_, _ = dist.min(dim=0)

_ = plt.hist(min_, bins=20)

In [ ]:
def repulsive_sampling(num_samples, loss_fn, device, max_iter=10000):
    lr = 1e-2

    X = torch.FloatTensor(num_samples, 3).uniform_(-1, 1)
    X = X.to(device)
    X.requires_grad_()

    for _ in range(max_iter):
        loss = rejection_loss(X, 1., 10.)
        loss.backward()

        with torch.no_grad():
            X -= lr * X.grad
            X.grad.zero_()

    X = X.detach().cpu()   
    X /= X.norm(dim=1, keepdim=True)
    
    return X[:,0], X[:,1], X[:,2]

def rejection_loss(x, alpha=1., beta=1.):
    N = x.shape[0]
    dist = (x.unsqueeze(0) - x.unsqueeze(1)).pow(2).sum(dim=2) + 1000 * torch.eye(N, N).to(x.device)   
    return alpha*(1/dist).mean() + beta*torch.abs(1. - x.norm(dim=1)).mean()
    

In [ ]:
def xyz2betagamma(x, y, z):           
    beta = torch.stack(
        (
            torch.atan2(-z, torch.sqrt(x.pow(2) + y.pow(2))), 
            torch.atan2(-z, torch.sqrt(x.pow(2) + y.pow(2))), 
        ),
        dim=1
    )
    
    gamma = torch.stack(
        (
            torch.atan2(-y, -x), 
            torch.atan2(y, x), 
        ),
        dim=1
    )
            
    mask = (-math.pi <= beta) & (beta < math.pi) & (-math.pi/2 <= gamma) & ( gamma < math.pi/2)
                        
    return beta[mask], gamma[mask]

def betagamma2xyz(beta, gamma):
    alpha, beta, gamma = alphabetagamma[:, 0], alphabetagamma[:, 1], alphabetagamma[:, 2]    
    x = (1 + alpha) * torch.cos(beta) * torch.cos(gamma) 
    y = (1 + alpha) * torch.cos(beta) * torch.sin(gamma) 
    z = -(1 + alpha) * torch.sin(beta)
    return x, y, z

In [ ]:
def xyz2alphabetagamma(x, y, z):

    alpha = torch.sqrt(x.pow(2) + y.pow(2) + z.pow(2)) - math.pi

    beta = torch.stack(
        (
            torch.atan2(-z, -torch.sqrt(x.pow(2) + y.pow(2))),
            torch.atan2(-z, torch.sqrt(x.pow(2) + y.pow(2))),
        ),
        dim=-1,
    )

    gamma = torch.stack((torch.atan2(-y, -x), torch.atan2(y, x)), dim=-1)

    mask = (beta >= -math.pi) & (beta < math.pi) & (gamma >= -math.pi/2) & (gamma < math.pi/2)

    return alpha, beta[mask], gamma[mask]

In [ ]:
import torch
import math

x = torch.tensor([-1.])
y = torch.tensor([0.])
z = torch.tensor([0.])

xyz2alphabetagamma(x, y, z)

In [ ]:
sign = lambda x: -1 if x < 0. else 1

In [ ]:
sign(0.)

In [ ]:
math.tan(math.fabs(math.pi/2 - beta/8))

In [ ]:
math.sgn(-1)

In [ ]:
beta = -math.pi/3
gamma = -math.pi/2
x = gamma
y = - 4 * math.tan(beta/4)
x, y

In [ ]:
math.log(1.)

In [ ]:
gamma = 0.
x = gamma
x

In [ ]:
nalpha = 2

alpha = torch.arange(0.0, math.pi, math.pi / nalpha).unsqueeze(1).repeat(1, 3)

xyz = (X.unsqueeze(1) * (1 + alpha.unsqueeze(0))).reshape(-1, 3)

In [ ]:
alphabetagamma = xyz2alphabetagamma(xyz)

In [ ]:
torch.allclose(xyz, alphabetagamma2xyz(alphabetagamma), atol=1e-6)

In [ ]:
def rejection_loss(x, alpha=1., beta=1.):
    N = x.shape[0]
    
    dist = (x.unsqueeze(0) - x.unsqueeze(1)).pow(2).sum(dim=2) + 1000 * torch.eye(N, N).to(x.device)
        
    norm = x.norm(dim=1)
    
    return alpha*(1/dist).mean() + beta*torch.abs(1. - norm).mean()

In [ ]:
fig = plt.figure(figsize=(8.0, 8.0))

ax = fig.add_subplot(
    111,
    projection="3d",
    xlim=(-1., 1.),
    ylim=(-1., 1.),
    zlim=(-1., 1.)
)

im = ax.scatter(
    X[:, 0],
    X[:, 1],
    X[:, 2]
)

# Mercator projection

In [ ]:
def mercator_projection(beta, gamma, images):
    ix = beta
    iy = torch.log(torch.tan(gamma/2 + math.pi/4))
    
    ix_ = (ix - ix.min())/(ix.max()-ix.min())
    iy_ = (iy - iy.min())/(iy.max()-iy.min())
    
    ix_inf = ix_.floor().long()
    ix_sup = ix_.ceil().long()
    iy_inf = iy_.floor().long()
    iy_sup = iy_.ceil().long()
    
    return (images[:,:,ix_inf, iy_inf] + images[:,:,ix_sup, iy_sup])/2

In [ ]:
num_samples = 100
nalpha = 3

In [ ]:
x, y, z = repulsive_spherical_sampling(num_samples, torch.device("cuda"), 20000)

In [ ]:
beta, gamma = xyz2betagamma(x, y, z)
alpha = torch.arange(0.0, math.pi, math.pi / nalpha)

In [ ]:
node_pos = torch.stack(
    (
        beta.unsqueeze(0).expand(nalpha, num_samples).flatten(), 
        gamma.unsqueeze(0).expand(nalpha, num_samples).flatten(), 
        alpha.unsqueeze(1).expand(nalpha, num_samples).flatten()
    )
)

In [ ]:
img = torch.rand(10, 3, 10, 10)

In [ ]:
img = mercator_projection(beta, gamma, img) 

In [ ]:
img = img.repeat(1, 1, nalpha)

In [ ]:
fig = plt.figure(figsize=(8.0, 8.0))

ax = fig.add_subplot(
    111,
    projection="3d",
)

im = ax.scatter(
    x, 
    y,
    z,
    c = img[0,0,0:100]
)

In [ ]:
fig = plt.figure(figsize=(8.0, 8.0))

ax = fig.add_subplot(
    111,
    projection="3d",
)

im = ax.scatter(
    x, 
    y,
    z,
    c = img[0, 0, 100:200]
)

In [ ]:
100*100*6

In [ ]:
import torch

x = torch.rand(100000,3)

In [ ]:
(x.unsqueeze(0) - x.unsqueeze(1)).pow(2).sum(dim=2)

In [ ]:
%timeit (x[0] - x[1:]).pow(2).sum(dim=1).topk(32, largest=False)

In [ ]:
RX = torch.tensor([
                [1, 0, 0],
                [0, cos(roll), -sin(roll)],
                [0, sin(roll), cos(roll)]
            ],requires_grad=True)

In [ ]:
def rotation_matrice(theta, axis):
    cos = torch.cos(theta)
    sin = torch.sin(theta)
    
    if axis == 0:
        return torch.tensor([
                [1, 0, 0],
                [0, cos, -sin],
                [0, sin, cos]
            ])
    
    if axis == 1:
        return torch.tensor([
                [cos, 0, sin],
                [0, 0, 0],
                [-sin, 0, cos]
            ])
    
    if axis == 2:
        return torch.tensor([
                [cos, -sin, 0],
                [sin, cos, 0],
                [0, 0, 0]
            ])

In [ ]:
theta = torch.rand(100)
theta.shape

In [ ]:
rotation_matrice(theta, 2)

In [ ]:
import torch

a = torch.tensor([[0., -1., 5.],[1., 0., 8.],[0., 0., 1.]])

In [ ]:
def se2group2matrix(g):
    N, D = g.shape
    
    cos = torch.cos(g[:,2])
    sin = torch.sin(g[:,2])
    
    Gg = torch.zeros(N, D, D)
    Gg[:, 0, 0] = cos
    Gg[:, 0, 1] = - sin
    Gg[:, 0, 2] = g[:,0]
    Gg[:, 1, 0] = sin
    Gg[:, 1, 1] = cos
    Gg[:, 1, 2] = g[:,1]
    Gg[:, 2, 2] = 1.

def matrix2se2group(Gg):
    

In [ ]:
def tensor_log(A, max_order=10):
    D = A.shape[-1]
    A = A - torch.eye(D)
    Ak = A.clone()
    
    M = torch.zeros(A.shape)
        
    for k in range(1, max_order):
        Ak = torch.matmul(Ak, A)
        if k % 2:
            M += Ak/k
        else:
            M -= Ak/k
            
        print(Ak, M)
            
    return M

In [ ]:
a = torch.rand(100, 3, 3)

In [ ]:
val, vec = torch.eig(a, eigenvectors=True)

In [ ]:
from scipy.linalg import logm

In [ ]:
theta = math.pi
x = 1
y = 1

a = torch.tensor([
    [math.cos(theta), -math.sin(theta), x],
    [math.sin(theta), math.cos(theta), y],
    [0, 0, 1]
])

In [ ]:
%%timeit
logm(a)

In [ ]:
log = torch.log(torch.complex(val[:,0], val[:,1]))

In [ ]:
vec[:,2]

In [ ]:
true_vec = torch.complex(vec[:,1], vec[:,2])
true_vec

In [ ]:
torch.complex(vec, torch.zeros(vec.shape)) @ log @ torch.complex(vec, torch.zeros(vec.shape)).T

In [ ]:
tensor_log(a, 100)

In [ ]:
a = torch.tensor([[-1.,  1.,  5.],
        [-1., -1.,  8.],
        [ 0.,  0.,  0.]])

In [ ]:
a @ a

In [ ]:
import torch
tensor = torch.rand(100, 3)

In [ ]:
a, b, c = tensor[:, [0,1,2]]